In [16]:
import numpy as np
import pandas as pd
import plotly.express as px

For prediction: chose two similarly-sized neighborhoods (rectangles) with very different location, population density, population non-white, adult asthma prevalence, life expectancy, social vulnerabilty index, and prevalence of emissions point sources.

In [17]:
# Neighborhood 1
n1_top_lat = 29.686200
n1_bot_lat = 29.672427
n1_right_lon = -95.319700
n1_left_lon = -95.367249

# Neighborhood 2
n2_top_lat = 29.763409
n2_bot_lat = 29.750580
n2_right_lon = -95.486371
n2_left_lon = -95.526338

In [18]:
n1_top_lat - n1_bot_lat

0.01377300000000048

In [19]:
n2_top_lat-n2_bot_lat

0.01282899999999998

In [20]:
n1_right_lon - n1_left_lon

0.047549000000003616

In [21]:
n2_right_lon - n2_left_lon

0.039966999999990094

In [22]:
print((n1_top_lat - n1_bot_lat)/10, "n1 N-S step")
print((n1_right_lon - n1_left_lon)/40, "n1 E-W step")
print((n2_top_lat-n2_bot_lat)/9, "n2 N-S step")
print((n2_right_lon - n2_left_lon)/34, "n2 E-W step")

0.0013773000000000478 n1 N-S step
0.0011887250000000905 n1 E-W step
0.001425444444444442 n2 N-S step
0.0011754999999997087 n2 E-W step


In [23]:
# create grid
# n1
n1_lats = np.linspace(n1_bot_lat, n1_top_lat, 10)
n1_lons = np.linspace(n1_right_lon, n1_left_lon, 40)

# n2
# n1
n2_lats = np.linspace(n2_bot_lat, n2_top_lat, 9)
n2_lons = np.linspace(n2_right_lon, n2_left_lon, 34)

In [24]:
lats = []
lons = []
for lat in n1_lats:
    for lon in n1_lons:
        lats.append(lat)
        lons.append(lon)
print(len(lats))
print(len(lons))
n1_dict = {"lat": lats, "lon": lons}

400
400


In [25]:
lats = []
lons = []
for lat in n2_lats:
    for lon in n2_lons:
        lats.append(lat)
        lons.append(lon)
print(len(lats))
print(len(lons))
n2_dict = {"lat": lats, "lon": lons}

306
306


In [26]:
df_n1 = pd.DataFrame(n1_dict)
df_n1 = df_n1.round(6)
df_n1

,lat,lon
0,29.672427,-95.319700
1,29.672427,-95.320919
2,29.672427,-95.322138
3,29.672427,-95.323358
4,29.672427,-95.324577
...,...,...
395,29.686200,-95.362372
396,29.686200,-95.363591
397,29.686200,-95.364811
398,29.686200,-95.366030


In [27]:
df_n2 = pd.DataFrame(n2_dict)
df_n2 = df_n2.round(6)
df_n2

,lat,lon
0,29.750580,-95.486371
1,29.750580,-95.487582
2,29.750580,-95.488793
3,29.750580,-95.490004
4,29.750580,-95.491215
...,...,...
301,29.763409,-95.521494
302,29.763409,-95.522705
303,29.763409,-95.523916
304,29.763409,-95.525127


In [28]:
df_predict = df_n1.append(df_n2)
df_predict

/var/folders/w0/hp1p0mz546x0rnc9mft_s83w0000gn/T/ipykernel_57563/3246832723.py:1: FutureWarning:

The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.



,lat,lon
0,29.672427,-95.319700
1,29.672427,-95.320919
2,29.672427,-95.322138
3,29.672427,-95.323358
4,29.672427,-95.324577
...,...,...
301,29.763409,-95.521494
302,29.763409,-95.522705
303,29.763409,-95.523916
304,29.763409,-95.525127


In [29]:
# neighborhood 1
px.set_mapbox_access_token("pk.eyJ1IjoidmlzaGFsdmpvc2VwaCIsImEiOiJjbDM2Y3hxaTMzYnlsM2JwZmoxZm9sZjZxIn0.ARpv_l4hUbukED2DPkPuxg")
fig = px.scatter_mapbox(df_predict, lat="lat", lon="lon", size_max=1, zoom=10)
fig.show()

In [30]:
# Export
filepath = "../../data/cleaned/predict.csv"
df_predict.to_csv(filepath)